# This notebook is used to process the gantry data and save it as a .pt file

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import csv
from pathlib import Path

## load data

In [ ]:
## Params
NUM_POS = 4096
DARK_THRESHOLD = None

# load data from csv file
raw_data = np.loadtxt('gantry_data_1.csv', delimiter=',',usecols=(2,3,4,5))
data_valid = np.array([_ for _ in raw_data if _ > DARK_THRESHOLD])

In [17]:
# caluculate resolution in 18 meters, beam moving resolution
import math
math.tan((1/3517)*280e-6)*18e3

0.001433039522320162

In [ ]:
# plot the data
plt.figure(figsize=(10,10))
for i in range(4):
    plt.scatter(range(len(data_valid)),data_valid[:,i],label=f'diod_{i+1}')
plt.legend()
plt.show()

## Form train and valida data set

In [27]:
# Form train set
#### at each position in ((-8,8)step size 0.25, total 64*64 positions), random sampling 3 data points
## form train input set
train_input = data_valid[]
## form train target set
# make a list of number from -8 to 8 at step size 0.25, and each of the number repeat 3 times
train_target = np.repeat(np.arange(-8,8,0.25),3)

In [30]:
# Form valida set
####at each position in ((-8,8)step size 0.25, total 64*64 positions), random sampling 1 data point
val_input = np.zeros((64,64,4))
val_target = np.arange(-8,8,0.25)

In [28]:
train_target

array([-8.  , -8.  , -8.  , -7.75, -7.75, -7.75, -7.5 , -7.5 , -7.5 ,
       -7.25, -7.25, -7.25, -7.  , -7.  , -7.  , -6.75, -6.75, -6.75,
       -6.5 , -6.5 , -6.5 , -6.25, -6.25, -6.25, -6.  , -6.  , -6.  ,
       -5.75, -5.75, -5.75, -5.5 , -5.5 , -5.5 , -5.25, -5.25, -5.25,
       -5.  , -5.  , -5.  , -4.75, -4.75, -4.75, -4.5 , -4.5 , -4.5 ,
       -4.25, -4.25, -4.25, -4.  , -4.  , -4.  , -3.75, -3.75, -3.75,
       -3.5 , -3.5 , -3.5 , -3.25, -3.25, -3.25, -3.  , -3.  , -3.  ,
       -2.75, -2.75, -2.75, -2.5 , -2.5 , -2.5 , -2.25, -2.25, -2.25,
       -2.  , -2.  , -2.  , -1.75, -1.75, -1.75, -1.5 , -1.5 , -1.5 ,
       -1.25, -1.25, -1.25, -1.  , -1.  , -1.  , -0.75, -0.75, -0.75,
       -0.5 , -0.5 , -0.5 , -0.25, -0.25, -0.25,  0.  ,  0.  ,  0.  ,
        0.25,  0.25,  0.25,  0.5 ,  0.5 ,  0.5 ,  0.75,  0.75,  0.75,
        1.  ,  1.  ,  1.  ,  1.25,  1.25,  1.25,  1.5 ,  1.5 ,  1.5 ,
        1.75,  1.75,  1.75,  2.  ,  2.  ,  2.  ,  2.25,  2.25,  2.25,
        2.5 ,  2.5 ,

# Real gantry data processing

## load data and visualize

fix the raw file and aplit line not enough data issue

In [23]:
# Define paths
data_root_path = Path.cwd().parent/'data'
raw_data_path = data_root_path/'pd_data'/'gantry_0213_160012.csv'

# Read and process the file
lines = raw_data_path.read_text().splitlines()[1:]  # skip the first headerline
modified_lines = []

for line in lines:
    line_list = line.split(',')
    if line_list[1].startswith('s') or line_list[1].startswith('W'):
        line = f"{line.strip()},0.0,0.0,0.0,0.0"
    modified_lines.append(line)

# Write back to file
raw_data_path.write_text('\n'.join(modified_lines))
# raw_data = np.loadtxt(raw_data_path, delimiter=',',skiprows=1,usecols=range(2,10))


1872498

In [72]:
data_root_path = Path.cwd().parent/'data'

pd_data_path = data_root_path/'pd_data'
gantry_data_path = data_root_path/'gantry_pos_data'

pd_data = []
for file in Path(pd_data_path).glob('gantry*.csv'):
    # go through all the line in file, if the 
    # split the line by comma and convert the string to int
    line_data = []
    for line in file.read_text().split('\n'):
        line = line.split(',')
        if line[1] in ['s','W']:
            continue
        line_data.extend([int(float(x)) for x in line[2:]])
        if line[1] == 'G4':
            pd_data.append(line_data)
            line_data = []
    print(len(pd_data))


6603
13192


In [73]:
len(pd_data[1])

32